In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.execution.adaptive.{AdaptiveSparkPlanExec}

Compile Error: <console>:27: error: object adaptive is not a member of package org.apache.spark.sql.execution
       import org.apache.spark.sql.execution.adaptive.{AdaptiveSparkPlanExec}
                                             ^


In [2]:
val sparkconf = new SparkConf()
    .setMaster("local[*]")
    .setAppName("Dynamic Join Startegy")

Compile Error: <console>:25: error: not found: type SparkConf
       val sparkconf = new SparkConf()
                           ^


In [3]:
val spark = SparkSession
    .builder()
    .config(sparkconf)
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB")
    .config("spark.sql.adaptive.logLevel", "TRACE")
    .getOrCreate()

Compile Error: <console>:27: error: not found: value sparkconf
           .config(sparkconf)
                   ^


In [4]:
val df1 = spark.sql(
    """
      |SELECT id AS i_item_id,
      |CAST(rand() * 1000 AS INT) AS i_price
      |FROM RANGE(30000000)
      |""".stripMargin
  )

df1 = [i_item_id: bigint, i_price: int]


[i_item_id: bigint, i_price: int]

In [5]:
df1.show(5)

+---------+-------+
|i_item_id|i_price|
+---------+-------+
|        0|     15|
|        1|    608|
|        2|    886|
|        3|    779|
|        4|    459|
+---------+-------+
only showing top 5 rows



In [6]:
df1.createOrReplaceTempView("items")

In [7]:
val df2 = spark.sql(
    """
      |SELECT CASE WHEN rand() < 0.8 THEN 100 ELSE CAST(rand() * 30000000 AS INT) END AS s_item_id,
      |CAST(rand() * 100 AS INT) AS s_quantity,
      |DATE_ADD(current_date(), - CAST(rand() * 360 AS INT)) AS s_date
      |FROM RANGE(10000000)
      |""".stripMargin
  )

df2 = [s_item_id: int, s_quantity: int ... 1 more field]


[s_item_id: int, s_quantity: int ... 1 more field]

In [8]:
df2.show(5)

+---------+----------+----------+
|s_item_id|s_quantity|    s_date|
+---------+----------+----------+
|      100|        78|2019-12-21|
|      100|        32|2020-06-22|
|      100|        85|2020-04-27|
|      100|        76|2020-03-25|
|      100|        23|2020-04-23|
+---------+----------+----------+
only showing top 5 rows



In [9]:
df2.createOrReplaceTempView("sales")

In [10]:
val dfAdaptive = spark.sql(
    """
      |SELECT s_date, sum(s_quantity * i_price) AS total_sales
      |FROM sales
      |JOIN items ON s_item_id = i_item_id
      |WHERE i_price < 10
      |GROUP BY s_date
      |ORDER BY total_sales DESC
      |""".stripMargin)

dfAdaptive = [s_date: date, total_sales: bigint]


[s_date: date, total_sales: bigint]

In [11]:
val planbefore = dfAdaptive.queryExecution.executedPlan

planbefore = 


*(7) Sort [total_sales#29L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(total_sales#29L DESC NULLS LAST, 200)
   +- *(6) HashAggregate(keys=[s_date#14], functions=[sum(cast((s_quantity#13 * i_price#1) as bigint))], output=[s_date#14, total_sales#29L])
      +- Exchange hashpartitioning(s_date#14, 200)
         +- *(5) HashAggregate(keys=[s_date#14], functions=[partial_sum(cast((s_quantity#13 * i_price#1) as bigint))], output=[s_date#14, sum#35L])
            +- *(5) Project [s_quantity#13, s_date#14, i_price#1]
               +- *(5) SortMergeJoin [cast(s_item_id#12 as bigint)], [i_item_id#0L], Inner
                  :- *(2) Sort [cast(s_item_id#12 as bigint) ASC NULLS FIRST], false, 0
                  :  +- Exchange ha...


In [12]:
val result = dfAdaptive.collect()

result = Array([2019-11-07,21851], [2020-02-10,18394], [2020-05-15,18382], [2020-04-25,18055], [2019-10-20,17096], [2020-06-07,16951], [2020-03-22,16944], [2020-03-01,16891], [2019-12-19,16889], [2020-06-28,16767], [2020-04-17,16765], [2020-05-30,16684], [2020-06-03,16682], [2020-05-02,16582], [2019-09-15,16582], [2019-12-31,16509], [2019-12-07,16403], [2020-07-22,16364], [2020-03-25,16271], [2020-08-06,16267], [2020-06-22,16111], [2019-11-11,16020], [2020-07-19,15991], [2019-12-08,15985], [2019-11-13,15977], [2020-04-09,15968], [2020-07-17,15921], [2020-08-11,15915], [2019-10-21,15888], [2019-09-08,15820], [2020-03-10,15769], [2020-02-15,15746], [2020-05-31,15517], [2020-06-12,15487], [2020-05-03,15472], [2020-05-09,15443], [2019-11-10,15425], [2020-06-...


Array([2019-11-07,21851], [2020-02-10,18394], [2020-05-15,18382], [2020-04-25,18055], [2019-10-20,17096], [2020-06-07,16951], [2020-03-22,16944], [2020-03-01,16891], [2019-12-19,16889], [2020-06-28,16767], [2020-04-17,16765], [2020-05-30,16684], [2020-06-03,16682], [2020-05-02,16582], [2019-09-15,16582], [2019-12-31,16509], [2019-12-07,16403], [2020-07-22,16364], [2020-03-25,16271], [2020-08-06,16267], [2020-06-22,16111], [2019-11-11,16020], [2020-07-19,15991], [2019-12-08,15985], [2019-11-13,15977], [2020-04-09,15968], [2020-07-17,15921], [2020-08-11,15915], [2019-10-21,15888], [2019-09-08,15820], [2020-03-10,15769], [2020-02-15,15746], [2020-05-31,15517], [2020-06-12,15487], [2020-05-03,15472], [2020-05-09,15443], [2019-11-10,15425], [2020-06-...

In [13]:
val planAfter = dfAdaptive.queryExecution.executedPlan

planAfter = 


*(7) Sort [total_sales#29L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(total_sales#29L DESC NULLS LAST, 200)
   +- *(6) HashAggregate(keys=[s_date#14], functions=[sum(cast((s_quantity#13 * i_price#1) as bigint))], output=[s_date#14, total_sales#29L])
      +- Exchange hashpartitioning(s_date#14, 200)
         +- *(5) HashAggregate(keys=[s_date#14], functions=[partial_sum(cast((s_quantity#13 * i_price#1) as bigint))], output=[s_date#14, sum#35L])
            +- *(5) Project [s_quantity#13, s_date#14, i_price#1]
               +- *(5) SortMergeJoin [cast(s_item_id#12 as bigint)], [i_item_id#0L], Inner
                  :- *(2) Sort [cast(s_item_id#12 as bigint) ASC NULLS FIRST], false, 0
                  :  +- Exchange has...


In [ ]:
dfAdaptive.show(100,false)